In [2]:
import cv2
import numpy as np
from timeit import default_timer as timer
import datetime
import math
import os

# Settings
video_addr='/home/unumkam/Desktop/repos/data/2020-12-29 12-22-31.181.wmv'
turn_on_delay = 5
x,y = 200,190
w,h = 1070,150
frames_limit = 10
detection_zones = 6
bound_velocity = 0.006 #8

# Common memory
measures = [ [ 0 for i in range(detection_zones) ] for j in range(frames_limit) ]
errorDelayCounters = [0]*detection_zones


def printLabel( image, label ):
    font = cv2.FONT_HERSHEY_SIMPLEX 
    # org 
    org = (50, 50) 
    # fontScale 
    fontScale = 1
    # Blue color in BGR 
    color = (255, 0, 0) 
  
    # Line thickness of 2 px 
    thickness = 2
    return cv2.putText(image, label, org, font,  
                   fontScale, color, thickness, cv2.LINE_AA)

def process_zone( zone_index, image):
    array=measures[zone_index]
    wasFound = False
    speedFactor = 0.0
    widthFactor = 0.0
    fullArea = image.shape[0]*image.shape[1]
    area = cv2.countNonZero(image)
    ratio = area/fullArea
    if( len(array) > frames_limit):
        measures[zone_index].pop(0)
    measures[zone_index].append(ratio)
    array=measures[zone_index]
    f_min = min(array)
    f_max = max(array)
    diffs = []
    if( len(array) == 1):
        return False,0,0
    for index in range(0,len(array)-1):
        diffs.append(math.fabs(array[index]-array[index+1]))
    if( max(diffs) > bound_velocity ):
        wasFound = True
    return wasFound,  max(diffs), f_max-f_min
            
        
def detect_artifacts(binarized_image,zones_count,x,y,w,h):
    answer = []
    for zone_index in range(0,zones_count):
        zone_width = int(w/zones_count)
        rect_x = x + zone_index*zone_width
        cropped_zone = binarized_image[y+1:y+h-2,rect_x:rect_x+zone_width-2]
        wasFound,diff_rate,amp_rate=process_zone(zone_index, cropped_zone)
        answer.append((wasFound,diff_rate))
    return answer

def addZoneRect(color_image, zone_index, zones_count, green_or_red, x,y,w,h ):
    zone_width = int(w/zones_count)
    rect_x = x + zone_index*zone_width
    color = ((1,255,1),(1,1,255))[green_or_red]
    dept=(4,1)[green_or_red]
    return cv2.rectangle(color_image,(rect_x,y),(rect_x+zone_width,y+h),color,dept)
    
    
def process_video_file():
    # Focusing delay
    start = timer()
    error = False
    errorText = ""
        
    cap = cv2.VideoCapture(video_addr)
    if not cap.isOpened():
        return True, "Error opening video stream or file"
    
    images = []
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == False:
            break
        gr_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret,binarizedImage = cv2.threshold(gr_img,215,255,cv2.THRESH_TOZERO)
        color_img = cv2.cvtColor(binarizedImage, cv2.COLOR_GRAY2RGB)
        
        images.append(gr_img)
        if( len(images)>3 ):
            images.pop(0)
            
        if( len(images)==3):      
            # im_t is the frame of interest; im_tp1 and im_tm1 are, respectively
            # the successive and previous frames.
            dbp = cv2.absdiff(images[1], images[0])
            db0 = cv2.absdiff(images[2], images[0])
            dbm = cv2.absdiff(images[1],images[2])
            detect = cv2.bitwise_not( cv2.bitwise_and(cv2.bitwise_and(dbp, dbm),cv2.bitwise_not(db0)))
            r,bb = cv2.threshold(detect,215,255,cv2.THRESH_TOZERO_INV)
            thresh_frame = cv2.dilate(bb, None, iterations = 2) 
            cv2.imshow('Original44',bb)
            cv2.imshow('Original46',thresh_frame)
            binarizedImage = thresh_frame
        
        is_ready =  timer() - start > turn_on_delay
        
        label = ""
        if is_ready:
            answer_metrics_pairs = detect_artifacts( binarizedImage, detection_zones,x,y,w,h)
            for zone_index in range(detection_zones):
                (answer,metrics)=answer_metrics_pairs[zone_index]
                label = label +("Y","N")[answer]+" "+"{:.5f}".format(metrics)+" "
                longError =  errorDelayCounters[zone_index] > 0
                if longError:
                    answer = True
                    errorDelayCounters[zone_index] =  errorDelayCounters[zone_index] - 1
                elif answer:
                    errorDelayCounters[zone_index] = 4
                color_img = addZoneRect( color_img, zone_index, detection_zones, answer, x,y,w,h)
        else:
            label = "focusing delay"
        
        finalImage = printLabel(color_img, label)
        cv2.imshow('Original',frame)
        cv2.imshow('Frame',finalImage)
                
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # When everything done, release the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()
    return False,""

process_video_file()


(False, '')

In [50]:
# Open camera

import numpy as np
import cv2

cap = cv2.VideoCapture(0)
while(True):
    cap.set(7, 0)
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 